# Day 09. Exercise 02
# Metrics

## 0. Imports

In [2]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from joblib import dump, load
from tqdm.notebook import trange, tqdm
import numpy as np

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [3]:
df = pd.read_csv("../data/dayofweek-not-scaled.csv")
X = df.drop(['dayofweek'], axis=1).values
y = df[['dayofweek']].values[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)
df.head()

,numTrials,hour,dayofweek,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,...,labname_lab02,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1
0,1,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## 2. SVM

1. Use the best parameters from the previous exercise and train the model of SVM.
2. You need to calculate `accuracy`, `precision`, `recall`, `ROC AUC`.

 - `precision` and `recall` should be calculated for each class (use `average='weighted'`)
 - `ROC AUC` should be calculated for each class against any other class (all possible pairwise combinations) and then weighted average should be applied for the final metric
 - the code in the cell should display the result as below:

```
accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97878
```

In [6]:
svc = SVC(C=10, class_weight=None, gamma='auto', kernel='rbf', random_state=21, probability=True)
svc.fit(X_train, y_train)
predict = svc.predict(X_test)
print("accuracy is", accuracy_score(y_test, predict))
print("precision is", precision_score(y_test, predict, average='weighted'))
print("recall is", recall_score(y_test, predict, average='weighted'))
print("roc_auc is", roc_auc_score(y_test, svc.predict_proba(X_test), multi_class='ovo', average='weighted'))

accuracy is 0.8875739644970414
precision is 0.8926729169690374
recall is 0.8875739644970414
roc_auc is 0.9787793228216216


In [5]:
svc.predict_proba(X_test)

array([[0.03295653, 0.90147801, 0.01173772, ..., 0.00755426, 0.00588124,
        0.00956905],
       [0.01700934, 0.06089425, 0.00793216, ..., 0.01503727, 0.74650625,
        0.14218874],
       [0.02706577, 0.03021426, 0.00261161, ..., 0.00197637, 0.01745052,
        0.90092184],
       ...,
       [0.00835454, 0.00516479, 0.88968793, ..., 0.04112849, 0.00732644,
        0.01408322],
       [0.13395529, 0.80105234, 0.00141938, ..., 0.00306151, 0.00313389,
        0.00449626],
       [0.15601416, 0.00845964, 0.77725999, ..., 0.00749456, 0.00454102,
        0.01014228]])

## 3. Decision tree

1. The same task for decision tree

In [66]:
dtc = DecisionTreeClassifier(max_depth=21, class_weight='balanced', criterion='gini', random_state=21)
dtc.fit(X_train, y_train)
predict = dtc.predict(X_test)
print("accuracy is", accuracy_score(y_test, predict))
print("precision is", precision_score(y_test, predict, average='weighted'))
print("recall is", recall_score(y_test, predict, average='weighted'))
print("roc_auc is", roc_auc_score(y_test, dtc.predict_proba(X_test), multi_class='ovr', average='weighted'))

accuracy is 0.8846153846153846
precision is 0.8876518218623483
recall is 0.8846153846153846
roc_auc is 0.9363813806150745


## 4. Random forest

1. The same task for random forest.

In [68]:
rfc = RandomForestClassifier(max_depth=24, class_weight='balanced', criterion='entropy',n_estimators=100, random_state=21)
rfc.fit(X_train, y_train)
predict = rfc.predict(X_test)
print("accuracy is", accuracy_score(y_test, predict))
print("precision is", precision_score(y_test, predict, average='weighted'))
print("recall is", recall_score(y_test, predict, average='weighted'))
print("roc_auc is", roc_auc_score(y_test, rfc.predict_proba(X_test), multi_class='ovr', average='weighted'))

accuracy is 0.9260355029585798
precision is 0.9275374670957044
recall is 0.9260355029585798
roc_auc is 0.9910624546710822


## 5. Predictions

1. Choose the best model.
2. Analyze: for which `weekday` your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which `labname` and for which `users`.
3. Save the model.

In [170]:
worst_day = 1
worst_day_acc  = 1
for day in range(7):
    indexes = np.where(y_test == day)
    predict = rfc.predict(X_test[indexes])
    predict = np.round(predict, 0)
    error = accuracy_score(predict, y_test[indexes])
    if error < worst_day_acc:
        worst_day_acc = error  
        worst_day = day
print("Worst day:", worst_day, "Error:", worst_day_acc)

worst_user = -1
worst_user_acc  = 1
for user in range(2, 33):
    indexes = np.where(X[:,user] == 1)
    if len(indexes[0]) == 0:
        continue
    predict = rfc.predict(X[indexes])
    predict = np.round(predict, 0)
    error = accuracy_score(predict, y[indexes])
    if error < worst_user_acc:
        worst_user_acc = error  
        worst_user = user
print("Worst user:", df.columns[worst_user + 1], "Acc:", worst_user_acc)

worst_lab = -1
worst_lab_acc  = 1
for lab in range(32, len(df.columns) - 1):
    indexes = np.where(X[:,lab] == 1)
    if len(indexes[0]) == 0:
        continue
    predict = rfc.predict(X[indexes])
    predict = np.round(predict, 0)
    error = accuracy_score(predict, y[indexes])
    if error < worst_lab_acc:
        worst_lab_acc = error  
        worst_lab = lab
print("Worst lab:", df.columns[worst_lab + 1], "Acc:", worst_lab_acc)

Worst day: 0 Error: 0.7777777777777778
Worst user: uid_user_6 Acc: 0.8333333333333334
Worst lab: labname_lab03 Acc: 0.0


In [171]:
dump(rfc, "MyBestModel")

['MyBestModel']

## 6. Function

1. Write a function that takes a list of different models and a corresponding list of parameters (dicts) and returns a dict that contains all the 4 metrics for each model.

In [187]:
def function(models, params):
    ret = {}
    for i in range(len(models)):
        model = models[i](**params[i])
        model.fit(X_train, y_train)
        predict = model.predict(X_test)
        acc = accuracy_score(y_test, predict)
        prec = precision_score(y_test, predict, average='weighted')
        recall = recall_score(y_test, predict, average='weighted')
        roc_auc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr', average='weighted')
        ret[type(model).__name__] = {'accuracy': acc,
                                     'precision' : prec,
                                     'recall' : recall,
                                     'ROC AUC' : roc_auc}
    return ret

In [193]:
function([RandomForestClassifier,DecisionTreeClassifier], [{'max_depth': 24},
                    {'max_depth':21, 'class_weight':'balanced', 'criterion':'gini', 'random_state':21}])

{'RandomForestClassifier': {'accuracy': 0.9319526627218935,
  'precision': 0.9348049473335871,
  'recall': 0.9319526627218935,
  'ROC AUC': 0.9914974586761659},
 'DecisionTreeClassifier': {'accuracy': 0.8846153846153846,
  'precision': 0.8876518218623483,
  'recall': 0.8846153846153846,
  'ROC AUC': 0.9363813806150745}}